# Обработка персональных данных

Записи, относящиеся к каждому "участнику" проекта, собраны теперь в текстовых файлах. Для простоты будем называть их блогами. В них по-прежнему много лишних строк. Нужно извлечь из блогов следующую информацию:

1) имя автора;
2) информацию о его социальных связях, занятиях, месте жительства, возрасте и т.п.

Кроме того, нужно разбить блоги на отдельные записи, для каждой определить дату и, по возможности, дополнительные атрибуты (источник и т.п.).

Начнем с выявления имени автора. Оно, по предварительным наблюдениям, содержится в первой строке блога, начинающейся со знака `#`. Будем считывать файлы из каталога `data` и, проходя по нит построчно, находить такую строку и ее номер, чтобы не учитывать всё, что идет до нее (информация о проекте в целом, которая нас не интересует).

In [2]:
import glob
import pandas as pd

def find_name(fn, arr):
    with open(fn, 'r') as input:
       cnt = 0 
       for line in input:
           if line.startswith('#'):
                arr.append((fn.replace('data/', '').replace('.txt', ''), line.replace('#', '').strip(), cnt))
                break
           cnt +=1 

heroes = []

[find_name(fn, heroes) for fn in glob.glob("data/*.txt")[:5]]

heroes_frame = pd.DataFrame(heroes, columns=['file_name', 'person', 'first_line'])
heroes_frame.head()
                

,file_name,person,first_line
0,nikolaj-duhonin,Николай Духонин,50
1,zinaida-gippius,Зинаида Гиппиус,50
2,ivan-petrunkevich,Иван Петрункевич,50
3,dzhon-henberi-uilyams,Джон Хэнбери-Уильямс,50
4,mark-shagal,Марк Шагал,50


Теперь необходимо пройтись по строкам каждого файла и удалить те, которые не несут полезной информации, а также пометить те, которые указывают на даты постов, источники и т.п.

In [108]:
import re
import numpy as nm

trash_full_words = ['**__Без вымысла __**', '\-', 'Читать с начала', 'ВКонтакте', 'Фейсбук', 'Твиттер', 'ОК', 'Telegram', 'WhatsApp', 'Viber', 'Скопировать ссылку', 'Вы подписаны']
trash_start_words = ['Проект начинается 14 ноября', '**Проект 1917**', 'Подпишитесь на героя', 'Подписаться Вы']
trash_contains_words = ['Календарь героя']
trash_end_words = ['Поделиться']

def is_trash(txt):
    if txt in trash_words:
        return 'a'
    return 'b'

def pure_person(i, sr):
    file_name = heroes_frame.iloc[i]['file_name']
    file_name = 'data/{}.txt'.format(file_name)
    df = pd.read_csv(file_name, skiprows=sr, sep='|', header=None)
    df.columns = ['text']
    df['stripped'] = df['text'].str.strip()
    for word in trash_full_words:
        df = df.drop(df[df['stripped'].str.strip() == word].index)
    for word in trash_start_words:
        df = df.drop(df[df['stripped'].str.startswith(word)].index)
    for word in trash_contains_words:
        df = df.drop(df[df['stripped'].str.contains(word)].index)
    for word in trash_end_words:
        df = df.drop(df[df['stripped'].str.endswith(word)].index)
    print(df.head())

pure_person(0, 50)    

#for i in range(len(heroes_frame)):    

                                                 text  \
0                                 #  Николай Духонин    
16                                    Возраст: **40**   
20  Юрий Готье Историк, профессор кафедры русской ...   
21                                            Москва    
22                                           05.12.17   

                                             stripped  
0                                  #  Николай Духонин  
16                                    Возраст: **40**  
20  Юрий Готье Историк, профессор кафедры русской ...  
21                                             Москва  
22                                           05.12.17  
